In [2]:
import os
import openai
import sys

sys.path.append("../..")

# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())  # read local .env file

# openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = "sk-xx"

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
# What does the loaded documents look like?
print(f"type of doc: {type(documents)}", "\n")
print(f"the number of docs: {len(documents)}", "\n")
print(f"the type of the first doc: {type(documents[3])}")
print(f"The first doc itself: {documents[3]}")

type of doc: <class 'list'> 

the number of docs: 41 

the type of the first doc: <class 'llama_index.schema.Document'>
The first doc itself: Doc ID: d8f991be-aaeb-434e-b7c8-f8fd34c4c115
Text: PAGE 4Coding AI Is the New Literacy Today we take it for granted
that many people know how to read and write. Someday, I hope,  it will
be just as common that people know how to write code, specifically for
AI. Several hundred years ago, society didn’t view language literacy
as a necessary skill. A small  number of people learned to read and
wri...


In [5]:
# The document is now parsed as 41 separate pages.  If we merge them together it will be easier to work with.
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents([document], service_context=service_context)

In [7]:
query_engine = index.as_query_engine()

In [8]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can continue reading, taking courses, and talking to domain experts to come up with new ideas. It is also helpful to focus on a specific application area where machine learning has not yet been applied. This can give you the opportunity to explore unique and creative applications that no one else has done yet. Finally, you can develop a side hustle or personal project that may not initially be part of your job but can still stir your creative juices and strengthen bonds with collaborators.


## Evaluation setup using TruLens

In [9]:
eval_questions = []
with open("eval_questions.txt", "r") as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find part of AI challenging?


In [10]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [11]:
print(len(eval_questions))

11


In [12]:
from trulens_eval import Tru

tru = Tru()

tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [39]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [40]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [41]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_d1bedbca2d1254d321d470ac381c67b2,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_d1bedbca2d1254d321d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:32.093018"", ""...",2023-12-27T04:05:36.251264,1.0,0.95,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",4,2185,0.003361
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e8e8d8fca5a9e3f23ac1a9d803048323,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_e8e8d8fca5a9e3f23ac...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:36.586749"", ""...",2023-12-27T04:05:39.391579,1.0,0.00,1.00,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,2,1712,0.002612
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_723b03c80f5c5a9ac400e690a7613aa7,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_723b03c80f5c5a9ac40...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:39.775468"", ""...",2023-12-27T04:05:42.654252,1.0,0.00,0.25,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,2,1706,0.002599
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f97c42708dbb5dc64556dd83f95d3462,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_f97c42708dbb5dc6455...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:42.990384"", ""...",2023-12-27T04:05:45.096983,1.0,0.50,1.00,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,2,1676,0.002554
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_94470e64650f839b2de9274390e6afc2,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_94470e64650f839b2de...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:45.431139"", ""...",2023-12-27T04:05:47.821476,1.0,0.00,0.50,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,2,1668,0.002539


In [42]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


ModuleNotFoundError: No module named 'ipywidgets'

: 

In [13]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [14]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [15]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [16]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a18958ce2689c3a4cb91d70cf31f4e9f,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_a18958ce2689c3a4cb9...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T22:31:41.999378"", ""...",2023-12-19T22:31:49.889449,1.0,0.95,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",7,2197,0.003386
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e6beabc6b2b7fb40156bd1c57d8e4a27,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_e6beabc6b2b7fb40156...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T22:31:50.128850"", ""...",2023-12-19T22:31:53.676730,1.0,0.00,1.00,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,3,1720,0.002628
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b207ea7cbd11e8c7c91d6ee18f009ec1,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_b207ea7cbd11e8c7c91...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T22:31:53.912783"", ""...",2023-12-19T22:31:57.221659,1.0,0.00,0.25,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,3,1706,0.002599
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_bd2949ccec6a589746b71c79354caa8d,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_bd2949ccec6a589746b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T22:31:57.518078"", ""...",2023-12-19T22:32:00.567511,1.0,0.60,1.00,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,3,1676,0.002554
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_75c8f519dafd4b17e2250c9af7671182,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_75c8f519dafd4b17e22...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T22:32:00.792857"", ""...",2023-12-19T22:32:03.571586,1.0,0.00,0.50,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,2,1668,0.002539


In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [20]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [21]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

In [23]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json: 100%|██████████| 799/799 [00:00<00:00, 4.25MB/s]
model.safetensors: 100%|██████████| 1.11G/1.11G [00:24<00:00, 44.5MB/s]
tokenizer_config.json: 100%|██████████| 443/443 [00:00<00:00, 2.65MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 28.3MB/s]
tokenizer.json: 100%|██████████| 17.1M/17.1M [00:00<00:00, 76.2MB/s]
special_tokens_map.json: 100%|██████████| 279/279 [00:00<00:00, 988kB/s]


In [24]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, it is important to first identify and scope the project. Consider your career goals and choose a project that complements them. Ensure that the project is responsible, ethical, and beneficial to people. As you progress in your career, aim to work on projects that grow in scope, complexity, and impact over time. Building a portfolio of projects that shows skill progression can also be helpful. Additionally, there are resources available in the book that provide guidance on starting your AI job search and finding the right AI job for you.


In [25]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine, app_id="Sentence Window Query Engine"
)

In [26]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively with a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and focused on building AI for the benefit of all people can inspire and motivate individuals to do the same.
What is the importance of networking in AI?
Networking is important in AI because it allows individuals

In [27]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.981818,0.327273,0.825,2.090909,0.000846


In [36]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


FileNotFoundError: [Errno 2] No such file or directory: 'streamlit'

### 2. Auto-merging retrieval

In [29]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="merging_index"
)

In [30]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [31]:
# Sample query
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 3bb8a62b-5767-4028-93c6-2f6415995c4c.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 58b2b29c-8d4c-4fc8-b96c-d475afcd806d.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial. You should be able to explain your thought process and the value of your work to others. This will help others see the potential in your projects and trust you with resources for larger endeavors.


In [32]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(
    automerging_query_engine, app_id="Automerging Query Engine"
)

In [33]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7fb796cd90 is calling an instrumented method <function BaseQueryEngine.query at 0x7f82220b8f40>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7ff16c59d0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7fb796cd90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f811c1a93a0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7ff16c59d0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f7fb7a8cf90 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f822209fc40>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: fb0b961e-0181-48c6-90d0-ed882a12e2d6.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: fdbd36a7-7d80-4178-adfb-572ba7fea4df.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function BaseSynthesizer.synthesize at 0x7f82211f02c0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f82211f0ae0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function Refine.get_response at 0x7f82215e7240>. The path of this call may be incorrect.
Guessing path of new

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a community. Additionally, collaborating with others and being able to influence and be influenced by others is critical for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f82211f0ae0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function Refine.get_response at 0x7f82215e7240>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects and the difficulty in coming up with a plan for building a system when the time required to achieve the target accuracy is unknown. Additionally, technical challenges are also common in AI, and even experienced researchers and practitioners have struggled with them at some point.
> Merging 3 nodes into parent node.
> Parent node id: 63364766-68ea-43b8-b1da-f35e502bc7a9.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 17a48b23-1c6a-4bf8-be5b-ca90ae18a40f.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f82211f0ae0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function Refine.get_response at 0x7f82215e7240>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7fb796cd90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f811c1a93a0>. The path of this call may be incorrect.
Guessing path of n

Is it normal to find part of AI challenging?
Yes, it is normal to find parts of AI challenging. The text mentions that even accomplished individuals in the AI community have struggled with technical challenges at some point. The author encourages newcomers to not be discouraged by imposter syndrome and assures them that everyone has experienced difficulties in their AI journey.
> Merging 1 nodes into parent node.
> Parent node id: f3f5c72d-4382-4b2d-944d-e8bf08ce8d15.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: b24f3a00-321b-4518-afd5-9e810c9228f4.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 08d45c20-2e75-4373-baea-ee0028fd32bd.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent nod

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f82211f0ae0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f7fb796d090 is calling an instrumented method <function Refine.get_response at 0x7f82215e7240>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f7ff1ba83d0) using this function.


What is the right AI job for me?
The right AI job for you would depend on your specific skills, interests, and career goals. It is important to consider your background and experience in order to determine which role or industry would be the best fit for you. Additionally, it may be easier to transition into a new role or industry within AI rather than trying to switch both at the same time.


In [35]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.972727,0.3,0.862121,2.181818,0.000892
